In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
import requests
from sklearn.utils import resample
import math
import re
import time
import random
from sklearn.model_selection import train_test_split
import kshingle as ks
from sklearn.utils import shuffle

## Individual Assignment Computer Science COPY

Joost Alma
577925ja


# Goal: perform duplicate detection: Which product descriptions refer to the very same product.

Propose a scalable solution for the involved computations by reducing the number of comparisons using LHS
LHS: evaluate for efficacy (werkzaamheid) and efficiency

ModelID only for training

Use bootstrapping (with replacement) 63% training data, 37% test data
Take at least 5 bootstraps (robustness)


# Performance evaluation:
1. Average (across bootstrapps) F1-measure (harmonic mean between precision and recall)


Interpretation:
- TP: The pairs of products that are predicted to be duplicates and are real duplicates. 
- FP: is given by the pairs of products that are predicted to be duplicates but are real non-duplicates. 
- TN: is given by pairs of products that are predicted to be nonduplicates and are real non-duplicates. 
- FN: The pairs of products that are predicted to be non-duplicates but are real duplicates

# Importing dataset

In [2]:
with open("TVs-all-merged.json") as jsonFile:
    data = json.load(jsonFile)
    
#     jsonFile.close() #waarom nodig?
#     data = json.dumps(data, indent=2, sort_keys=True) #make data more readable

# Create list to be filled with datafile

In [59]:
file = []

##Add datafile to the list 'file'
for i in range(len(list(data.values()))):
    if len(list(data.values())[i]) ==1:
        file.append(list(data.values())[i][0])
    else:
        for j in range(len(list(data.values())[i])):
            file.append(list(data.values())[i][1:][0])
    
df = pd.DataFrame(file)
display(df)


,shop,url,modelID,featuresMap,title
0,bestbuy.com,http://www.bestbuy.com/site/Philips+-+4000+Ser...,29PFL4508/F7,"{'Screen Size Class': '29""', 'DVI Inputs': '0'...","Philips 4000 Series 29"" Class 2812"" Diag. LED ..."
1,newegg.com,http://www.newegg.com/Product/Product.aspx?Ite...,SC-3211,"{'Headphone': 'Yes', 'Remote Control': 'Full F...","Newegg.com - SuperSonic 32"" 720p LED HDTV SC-3211"
2,bestbuy.com,http://www.bestbuy.com/site/Sharp+-+AQUOS+-+90...,LC-90LE657U,"{'Language Options': 'English, French, Spanish...","Sharp AQUOS 90"" Class 90"" Diag. LED 1080p 120H..."
3,bestbuy.com,http://www.bestbuy.com/site/Philips+-+2000+Ser...,39PFL2908/F7,{'Meta Title': 'Philips Electronics 39PFL2908'...,"Philips 2000 Series 39"" Class 3858"" Diag. LED ..."
4,newegg.com,http://www.newegg.com/Product/Product.aspx?Ite...,LC70LE550U,"{'Remote Control': 'Yes', 'Series': 'Aquos', '...","Newegg.com - Sharp Aquos 70"" Class (69.5"" Diag..."
...,...,...,...,...,...
1619,bestbuy.com,http://www.bestbuy.com/site/Panasonic+-+VIERA+...,TC-P55GT31,"{'Internet Connectable': 'Yes', 'Language Opti...","Panasonic - VIERA 55"" Class / Plasma / 1080p ..."
1620,newegg.com,http://www.newegg.com/Product/Product.aspx?Ite...,UN40EH5000FXZA,"{'Component Video': '1 In', 'Energy Star Compl...","Samsung 40"" Class (40.0"" Diag.) 1080p 60Hz LED..."
1621,newegg.com,http://www.newegg.com/Product/Product.aspx?Ite...,UN40EH5000FXZA,"{'Component Video': '1 In', 'Energy Star Compl...","Samsung 40"" Class (40.0"" Diag.) 1080p 60Hz LED..."
1622,bestbuy.com,http://www.bestbuy.com/site/Insignia%26%23153%...,NS-32E859A11,"{'Internet Connectable': 'Yes', 'Language Opti...","Insignia - Connected TV 32"" Class / 1080p / 1..."


In [4]:
##extract all double modelID's from datafile
dupID = df.pivot_table(columns=['modelID'], aggfunc='size')
dupID = dupID[dupID > 1]
print("Total ammount of duplicates:", len(dupID))

##Enter a productID to check whether and how many times it is in the dataset 
def duplicateControl(A):
    if A in dupID:
        print(A, 'is contained in dupID', dupID[A], 'times')
    else:
        print(A, 'is not more than once in the dataset')
        
duplicateControl('22LN4510')


Total ammount of duplicates: 329
22LN4510 is contained in dupID 2 times


# Extract all words of the dataset and put in list to eliminate duplicates


# Bootstrapping, train & test datasets

In [5]:
##seperate the x_data and y_data

x_data = df[['shop', 'url', 'featuresMap', 'title']]
y_data = df['modelID']

In [6]:
##split x_data and y_data into x&y train & test data
##Traindataset is roughly 63% of the original dataset
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.37)
print('length xtrain', len(x_train)); print('length ytrain', len(y_train))
print('length xtest', len(x_test)); print('length ytest', len(y_test))

length xtrain 1023
length ytrain 1023
length xtest 601
length ytest 601


# Keywords & convert titles to words

Use title only for now

In [7]:
##First, make a function which converts the titles into a strings 
#https://www.learndatasci.com/tutorials/building-recommendation-engine-locality-sensitive-hashing-lsh-python/

##Function: input is the index number of the title to be converted into strings


def textfunc(A, index_sample):
    words = A.iloc[index_sample]
#     words = re.sub(r'[^\w\s]','', words)
    
    units = words.split()
    units = words.lower() #needed for modelID to be filtered
    return str(units)

print(textfunc(x_data['title'], 0)) #gives the title of the first product in x-trainset as strings
print(textfunc(y_data, 0))


philips 4000 series 29" class 2812" diag. led 720p 60hz hdtv 29pfl4508f7 - best buy
29pfl4508/f7


In [23]:
##create a list consisting of all modelID's to later filter them out of the titles for keywords
listIDs = []
for i in range(len(df)):
    listIDs.append(textfunc(y_data, i))  
    listIDs[i] = listIDs[i].replace('/', '')
    listIDs[i] = listIDs[i].replace('-', '')
listIDs[0:10]

['29pfl4508f7',
 'sc3211',
 'lc90le657u',
 '39pfl2908f7',
 'lc70le550u',
 'un40f6350a',
 'hg40na577lf',
 'un46f5000af',
 'e463',
 'un46es6580']

# Combine all titles
Create shingles from the titles

In [69]:
##create list to add all titles in string format
titles_string = []
for i in range(len(x_data)):
    titles_string.append(textfunc(x_data['title'], i))
    titles_string[i] = titles_string[i].replace('"', 'inch') #transform to inch
    titles_string[i] = titles_string[i].replace('inch diag', 'inch')
    titles_string[i] = titles_string[i].replace('inch.', 'inch') 
    titles_string[i] = titles_string[i].replace('smart hdtv', 'smartHdtv')
    titles_string[i] = titles_string[i].replace('smart led hdtv', 'smartLedHdtv')
    titles_string[i] = titles_string[i].replace('led hdtv', 'ledHdTv')
    titles_string[i] = titles_string[i].replace('hd tv', 'hdtv')
    titles_string[i] = titles_string[i].replace('ultra hdtv', 'ultraHdTv')

    titles_string[i] = titles_string[i].replace('smart 3d hdtv', 'smart3DHdtv')
    titles_string[i] = titles_string[i].replace('smart 3d plasma tv', 'smart3DPlasmatv')
    titles_string[i] = titles_string[i].replace(" hz", "hz") #transform to hz
    titles_string[i] = titles_string[i].replace("-", "")    


        
print(titles_string)



['philips 4000 series 29inch class 2812inch led 720p 60hz hdtv 29pfl4508f7  best buy', 'newegg.com  supersonic 32inch 720p ledHdTv sc3211', 'sharp aquos 90inch class 90inch led 1080p 120hz smart3DHdtv lc90le657u  best buy', 'philips 2000 series 39inch class 3858inch led 1080p 60hz smartHdtv 39pfl2908f7  best buy', 'newegg.com  sharp aquos 70inch class (69.5inchonal) 1080p 120hz ledlcd hdtv  lc70le550u', 'newegg.com  refurbished: samsung 40inch 1080p 120hz smartLedHdtv  un40f6350a', 'samsung 40inch class 40inch ledlcd tv 1080p hdtv 1080p hg40na577lf  best buy', 'samsung 46inch class 46inch ledlcd tv 1080p hdtv 1080p rose black un46f5000af  best buy', 'nec refurbished 46inch class 46inch ledlcd tv 1080p hdtv 1080p e463  best buy', 'newegg.com  refurbished: samsung 46inch class (45.9inch) 1080p 240hz ledHdTv un46es6580', 'newegg.com  refurbished: samsung 46inch class (45.9inch) 1080p 240hz ledHdTv un46es6580', 'tcl 39inch class 3812inch led 1080p 60hz hdtv le39fhde3010  best buy', 'toshib

# Use words in titles only

In [70]:
##Makes the titles of the products into a (by word callable list of strings)
def keyWordMaker(indexTitles):
    keyTitles = (titles_string[indexTitles])
    keyTitleSplit = keyTitles.split() 
    return keyTitleSplit

print(keyWordMaker(6))

['samsung', '40inch', 'class', '40inch', 'ledlcd', 'tv', '1080p', 'hdtv', '1080p', 'hg40na577lf', 'best', 'buy']


In [75]:
#add all individual lists of strings together in one list (callable by product index)
totalWords = []
for i in range(len(x_data)):
    totalWords.append(keyWordMaker(i))
print('Total amount of keywords including double hits of eg 720p:', len(totalWords))



#Remove all double keywords from totalWords
allKeyWords = []
for i in range(len(df)):
    for j in range(len(totalWords[i])):
        if totalWords[i][j] not in allKeyWords:
            allKeyWords.append(totalWords[i][j])
    if listIDs[i] in allKeyWords: ##remove modelID's that are in titles
        allKeyWords.remove(listIDs[i])        
# print(allKeyWords)


# for i in range(len(df)):
#     if listIDs[i] in allKeyWords:
#         allKeyWords.remove(listIDs[i])
    
allKeyWords.remove('class'); allKeyWords.remove('newegg.com')
allKeyWords.remove('best'); allKeyWords.remove('buy')


# print(totalWords[0])
print("Length keywords using nonduplicates from solely the titles of x_data:", len(allKeyWords))

# allKeyWords

Total amount of keywords including double hits of eg 720p: 1624
Length keywords using nonduplicates from solely the titles of x_data: 642


# Create matrices for input matrix

In [79]:
#make single list containing all keywords ['', '',  , ..., ''] --> allKeyWordsSep
print(len(allKeyWords)) #-->p
# allKeyWordsSep
print(len(totalWords))

642
1624


In [80]:
##create inputMatrix with binary vectors for each keyword (rows) in each product (columns)
inputMatrix = np.zeros((len(allKeyWords), len(x_data)))


for k in range(len(allKeyWords)):
    for i in range(len(x_data)):
            if allKeyWords[k] in totalWords[i]:
                    inputMatrix[k][i] = 1
            


In [81]:

print('Columns of inputmatrix (should equal #products):', len(inputMatrix[0]))
print('Length of inputmatrix (=rows (should equal #keywords)):', len(inputMatrix))
inputMatrix

Columns of inputmatrix (should equal #products): 1624
Length of inputmatrix (=rows (should equal #keywords)): 642


array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

## Minhasing


In [82]:
r = len(allKeyWords)
p = 1722 #needs to be strictly larger than k

In [84]:
# #correct minhash function
# def minHash(rowNumber):
#     a = random.randint(0, 100)
#     b = random.randint(0, 100)
#     h = (a + b * rowNumber) % p
#     return h

Now the real matrix

In [145]:
#Random number generator function with integer as input
def randomNumberGenerator(value):
    #create list to store random values such that random values are all created once when function is called
    randomNumbers = []
    indexValue = random.randint(0, len(allKeyWords) - 1)
    randomNumbers.append(indexValue)
    
    while value > 0:
        while indexValue in randomNumbers:
            indexValue = random.randint(0, len(allKeyWords) - 1)
        randomNumbers.append(indexValue)
        value=value-1
    return randomNumbers

numMinHash = 100
p = 1543

In [146]:
#In separate cell call random number generator function

randomIntA = randomNumberGenerator(numMinHash)
randomIntB = randomNumberGenerator(numMinHash)


0.0006401538848876953
0.0010330677032470703


In [87]:
signatureMatrix = []

for i in range(len(inputMatrix[0])): #first loop over all products
    signature = []

    for j in range(0, numMinHash): #Do this 100 times
        minHashValue = 2**32 - 1
        
        for k in range(len(allKeyWords)): #then loop over all keywords
            if inputMatrix[k][i] == 1:
            
                hashValue = (randomIntA[j] + k * randomIntB[j]) % p

                if hashValue < minHashValue:                    
                    minHashValue = hashValue 
                    
                    
        signature.append(minHashValue)     
    signatureMatrix.append(signature)

In [161]:
signatureMatrixFin = (pd.DataFrame(signatureMatrix))                    
print(len(signatureMatrixFin))
print(len(signatureMatrixFin[0])) #???



signatureMatrixFin


1624
1624


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,116,173,14,0,59,23,34,370,120,103,...,16,49,43,125,78,223,289,120,154,244
1,415,197,416,391,312,255,90,460,460,242,...,216,246,180,477,399,130,541,215,334,213
2,17,185,79,317,88,13,146,445,59,659,...,16,55,37,207,13,288,499,37,51,151
3,116,173,14,0,171,144,258,370,120,89,...,16,49,37,72,78,37,289,37,257,89
4,67,209,79,243,88,13,34,490,59,103,...,73,52,37,125,77,288,625,37,51,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619,317,6,79,227,237,94,34,147,94,103,...,15,52,13,125,11,139,625,37,49,151
1620,166,209,14,428,229,7,90,475,338,139,...,416,246,31,289,142,511,583,37,308,27
1621,166,209,14,428,229,7,90,475,338,139,...,416,246,31,289,142,511,583,37,308,27
1622,415,197,79,317,88,3,90,520,229,226,...,187,55,37,45,399,102,709,37,49,151


# Divide signature matrix into b bands and r rows for each band.
n = b * r, where n = length of columns of signature matrix M 

In [165]:
def threshold(r, b):
    if r * b == 100:
        check = str('r and b chosen correctly')
        t = (1/b) ** (1/r)
    return check, t

In [166]:
print(b*r)
b = 20
r = 5
threshold(r,b)
# threshold(r,b)[1]
# t = 0.8

100


('r and b chosen correctly', 0.5492802716530588)

In [153]:
# def jaccard(a: set, b: set):
#     return len(a.intersection(b)) / len(a.union(b))



In [151]:

for i in range(len(x_data)):
    for j in range(0, numMinHash):
        

# jaccard(a, b), jaccard(set(a_sig), set(b_sig))



TypeError: unsupported operand type(s) for |: 'str' and 'str'

In [ ]:
jaccard(a, c), jaccard(set(a_sig), set(c_sig))

In [ ]:
for i in range(len(titles_string)):
    signature1 = sigComplete[i]
    
    for j in range(i+1, len(titles_string)):
        signature2 = sigComplete[j]
        
        count = 0
        for k in range(0, len(removeDupl)):
            count = count + (signature1[k] == signature2[k])

In [ ]:
for i in range(0, numDocs):
  # Get the MinHash signature for document i.
  signature1 = signatures[i]
    
  # For each of the other test documents...
  for j in range(i + 1, numDocs):
    
    # Get the MinHash signature for document j.
    signature2 = signatures[j]
    
    count = 0
    # Count the number of positions in the minhash signature which are equal.
    for k in range(0, numHashes):
        count = count + (signature1[k] == signature2[k])
    
    # Record the percentage of positions which matched.    
    estJSim[getTriangleIndex(i, j)] = (count / numHashes)

In [ ]:
# numMinHash = 100
# sigComplete = []
# for i in range(len(inputMatrix[0])): #titles --> columns
#     sig = []
#     for j in range(numMinHash):
#         minHashValue = 2**32 - 1
#         for k in range(len(removeDupl)): #keywords --> rows
#             if inputMatrix[k][i] == 1:
#                 hashValue = (randomNoA[k] + k*randomNoB[k]) % p
#                 if hashValue < minHashValue:
#                     minHashValue = hashValue
        
#         sig.append(minHashValue)
#     sigComplete.append(sig)
    